In [ ]:
from pathlib import Path
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1 import make_axes_locatable
import h5py
import copy
import pickle

# Project modules
from lib.Stokes import Stokes
from functions.plot_data import plot_data

In [ ]:
# Check that the Stokes object file exists
stokes_filename = "generated/objects/stokes.pickle"
my_file = Path(stokes_filename)
if not my_file.is_file():
    print("Stokes object does not exists (", stokes_filename,"), please run file init_data.py first")

# Load the objects from the files
print("Loading Stokes data from file", stokes_filename)
stokes_list = pickle.load(open(stokes_filename, "rb"))

In [ ]:
# Calculate first derivative
stokes_list['I'].calc_first_derivative()

In [ ]:
# Calculate second derivative
stokes_list['I'].calc_second_derivative()

In [ ]:
# Plot derived Intensity
fig = stokes_list['I'].plot_frame_d(7)
fig = stokes_list['I'].plot_frame_d(25)

In [ ]:
# Plot derived Intensity
fig = stokes_list['I'].plot_frame_dd(7)
fig = stokes_list['I'].plot_frame_dd(25)

In [ ]:
plt.plot(stokes_list['I'].data_n[450,450,:])
plt.grid()

In [ ]:
plt.plot(stokes_list['I'].data_d[450,450,:],color='purple')
plt.grid()

In [ ]:
plt.plot(stokes_list['I'].data_dd[450,450,:],color='red')
plt.grid()

In [ ]:
# # Plot derived Intensity
# for i in range(117):
#     fig = stokes_list['I'].plot_frame_dd(i)

In [ ]:
# Calculate the vertical (longitudinal) component
C1 = 1
f = 1
Bll = -np.sum(stokes_list['V'].data_n * stokes_list['I'].data_d, 2) / (C1 * f * np.sum(stokes_list['I'].data_d*stokes_list['I'].data_d, 2))

In [ ]:
plot_data(Bll, f'B longitudinal')

In [ ]:
# Calculate the horizontal (transverse) component
C2 = 1 #TODO
L = np.sqrt(stokes_list['Q'].data_n**2 + stokes_list['U'].data_n**2)
Bt = np.sum(L * np.abs(stokes_list['I'].data_dd), 2) / (C2 * f * np.sum(np.abs(stokes_list['I'].data_dd), 2)*2)
Bt = np.sqrt(Bt)

In [ ]:
plot_data(-Bt, f'B transverse')

In [ ]:
# Compute the angle
tan2chi = np.sum(stokes_list['U'].data_n * stokes_list['I'].data_dd, 2) / np.sum(stokes_list['Q'].data_n * stokes_list['I'].data_dd, 2) 
chi = np.arctan(tan2chi)/2

In [ ]:
plot_data(chi, 'Chi') #TODO: this plot is not number of counts


In [ ]:
chi2 = (np.arctan2(np.sum(stokes_list['U'].data_n * stokes_list['I'].data_dd, 2), np.sum(stokes_list['Q'].data_n * stokes_list['I'].data_dd, 2))) / 2

In [ ]:
# Plot one frame of data
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(8, 8))
ax.set_xlabel('X axis (array index)')
ax.set_ylabel('Y axis (array index)')
ax.set_title('Chi')

img = ax.imshow(np.rad2deg(chi2), cmap='twilight')
    
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(img, cax=cax, label='Angle [degrees]')


#TODO: why is the result only to +/- 90 degrees? /2 factor

In [ ]:
# Calculate inclination angle
lambda0 = 1 #TODO
gbar = 1 #TODO
Gbar = 1 #TODO
temp = np.sum(np.abs(stokes_list['I'].wave_array - lambda0) * L * stokes_list['V'].data_n**2 * stokes_list['I'].data_d, 2)
tan2gamma = 4/3 * gbar**2 / Gbar * temp / np.sum(np.abs(stokes_list['V'].data_n)**4)
gamma = np.arctan(np.sqrt(tan2gamma))

In [ ]:
# Plot one frame of data
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(8, 8))
ax.set_xlabel('X axis (array index)')
ax.set_ylabel('Y axis (array index)')
ax.set_title('Chi')

img = ax.imshow(np.rad2deg(gamma), cmap='twilight')
    
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(img, cax=cax, label='Angle [degrees]')

In [ ]:
gamma = np.cotan(tan2gamma)